## NLP Assignment - 1 . Sentiment Analysis for Amazon product Reviews
Group 144
Members:

R B Khare BITS ID 2020FC04857

Anurag BITS ID 2020FA04028

Abhinav Chandravanshi BITS ID 2020FC04860

#### Data Set used amazon_reviews.csv downloaded from https://www.kaggle.com/rogate16/amazon-reviews-2018-full-dataset

#### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfTransformer

#### Reading in the Dataset

In [3]:
# Reading the dataset.
dataset = pd.read_csv('D:\\M Tech DSE WILP\\Sem 3\\NLP\\Assignment\\amazon_reviews.csv')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551159 entries, 0 to 551158
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userName     551098 non-null  object 
 1   verified     551159 non-null  bool   
 2   itemName     551158 non-null  object 
 3   description  551151 non-null  object 
 4   image        551159 non-null  object 
 5   brand        547290 non-null  object 
 6   feature      551159 non-null  object 
 7   category     551159 non-null  object 
 8   price        476825 non-null  object 
 9   rating       551159 non-null  float64
 10  reviewTime   551159 non-null  object 
 11  summary      550925 non-null  object 
 12  reviewText   550764 non-null  object 
 13  vote         551159 non-null  int64  
dtypes: bool(1), float64(1), int64(1), object(11)
memory usage: 55.2+ MB


#### Product reviews of only verified purchases are to be used. Hence the dataset is filetred to retain only these 

In [5]:
dataset = dataset[dataset['verified']==True]

In [6]:
dataset = dataset.drop('verified',axis = 1).reset_index(drop = True)

#### Define Prediction dataset of 10000 records in advance

In [7]:
prediction_dataset= dataset.iloc[40000:50000]

#### Define Training dataset of 40000 records

In [8]:
dataset = dataset.loc[:40000,['itemName','brand','category','rating','summary','reviewText']]

#### Review Number of Products

In [9]:
len(dataset['itemName'].value_counts())

22246

## Preprocessing data from training dataset

#### Converting Text to lower case and string trimming

In [10]:
dataset['summary'] = dataset['summary'].astype(str)

In [11]:
dataset['summary'] = dataset['summary'].str.lower().str.strip()

In [12]:
dataset['reviewText'] = dataset['reviewText'].str.lower().str.strip()

#### Labelling the records for training purpose based on numerical ratings. Label -1 for Ratings 1,2,3,4 (Low). Label 1 for Rating 5 (high). These labels will be used for training of review text to reflect high (positive) sentiment of lower (negative) sentiment 

In [13]:
for rating in np.sort(dataset['rating'].unique()):
    if rating in [1,2,3,4]:
        dataset.at[dataset[dataset['rating'] == rating].index.tolist(),'labels'] = -1
    if rating in [5]:
        dataset.at[dataset[dataset['rating'] == rating].index.tolist(),'labels'] = 1

In [14]:
dataset['labels'] = dataset['labels'].astype(int)

In [15]:
dataset

itemName            brand  \
0      Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...        Toblerone   
1      Prince of Peace Organic Tea, Oolong, 100 Tea Bags  Prince Of Peace   
2      Pet Champion Adjustable No-Pull Harness, Colla...     Pet Champion   
3      Parker Urban Premium Ebony Metal Chiseled, Bal...           Parker   
4      Merrick Whole Earth Farms Adult Recipe Dry Dog...          Merrick   
...                                                  ...              ...   
39996         Fluker's Ceramic Heat Emitter for Reptiles         Fluker's   
39997  Purina Friskies Pate Wet Cat Food - (24) 5.5 o...  Purina Friskies   
39998  PetSafe Healthy Pet Water Station, Dog and Cat...          PetSafe   
39999        Alumilite Amazing Mold Rubber Kit, 0.77 oz.        Alumilite   
40000  Our Pets Elevated Bone Feeder Raised Dog Feedi...         Our Pets   

                       category  rating  \
0                  Prime_Pantry     5.0   
1      Grocery_and_Gourmet_Food     5.0   
2                  Pet_Supplies     5.0   
3               Office_Products     5.0   
4                  Pet_Supplies     2.0   
...                         ...     ...   
39996              Pet_Supplies     5.0   
39997              Pet_Supplies     5.0   
39998              Pet_Supplies     1.0   
39999    Arts_Crafts_and_Sewing     5.0   
40000              Pet_Supplies     5.0   

                                                 summary  \
0                                             five stars   
1                                             five stars   
2      she has an odd shape chest and her pull over h...   
3                                        i love this pen   
4                                              no bueno.   
...                                                  ...   
39996                                         five stars   
39997                                        bon apptit!   
39998                                  rust spots!  ugh!   
39999                                         five stars   
40000                            exactly what i expected   

                                              reviewText  labels  
0               super smooth and yummy with crunchy bits       1  
1                                   perfect for kombucha       1  
2      finally a harness that fits my puppy. i really...       1  
3      i love this pen! i love the shape of it, the f...       1  
4      we accidentally purchased this instead of our ...      -1  
...                                                  ...     ...  
39996                                      working great       1  
39997  this is a small rescue shelter and we go throu...       1  
39998  i've only had this for a couple of months, and...      -1  
39999  i like this stuff think it's a little pricy bu...       1  
40000  this stand is much more attractive than the sm...       1  

[40001 rows x 7 columns]

#### Removing Punctuation from the Text

In [16]:
dataset['reviewText'] = dataset['reviewText'].astype(str)

In [17]:
for punc in list(string.punctuation):
    dataset['summary'] = dataset['summary'].apply(lambda x:x.replace(punc,' '))
    dataset['reviewText'] = dataset['reviewText'].apply(lambda x:x.replace(punc,' '))

In [18]:
dataset

itemName            brand  \
0      Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...        Toblerone   
1      Prince of Peace Organic Tea, Oolong, 100 Tea Bags  Prince Of Peace   
2      Pet Champion Adjustable No-Pull Harness, Colla...     Pet Champion   
3      Parker Urban Premium Ebony Metal Chiseled, Bal...           Parker   
4      Merrick Whole Earth Farms Adult Recipe Dry Dog...          Merrick   
...                                                  ...              ...   
39996         Fluker's Ceramic Heat Emitter for Reptiles         Fluker's   
39997  Purina Friskies Pate Wet Cat Food - (24) 5.5 o...  Purina Friskies   
39998  PetSafe Healthy Pet Water Station, Dog and Cat...          PetSafe   
39999        Alumilite Amazing Mold Rubber Kit, 0.77 oz.        Alumilite   
40000  Our Pets Elevated Bone Feeder Raised Dog Feedi...         Our Pets   

                       category  rating  \
0                  Prime_Pantry     5.0   
1      Grocery_and_Gourmet_Food     5.0   
2                  Pet_Supplies     5.0   
3               Office_Products     5.0   
4                  Pet_Supplies     2.0   
...                         ...     ...   
39996              Pet_Supplies     5.0   
39997              Pet_Supplies     5.0   
39998              Pet_Supplies     1.0   
39999    Arts_Crafts_and_Sewing     5.0   
40000              Pet_Supplies     5.0   

                                                 summary  \
0                                             five stars   
1                                             five stars   
2      she has an odd shape chest and her pull over h...   
3                                        i love this pen   
4                                              no bueno    
...                                                  ...   
39996                                         five stars   
39997                                        bon apptit    
39998                                  rust spots   ugh    
39999                                         five stars   
40000                            exactly what i expected   

                                              reviewText  labels  
0               super smooth and yummy with crunchy bits       1  
1                                   perfect for kombucha       1  
2      finally a harness that fits my puppy  i really...       1  
3      i love this pen  i love the shape of it  the f...       1  
4      we accidentally purchased this instead of our ...      -1  
...                                                  ...     ...  
39996                                      working great       1  
39997  this is a small rescue shelter and we go throu...       1  
39998  i ve only had this for a couple of months  and...      -1  
39999  i like this stuff think it s a little pricy bu...       1  
40000  this stand is much more attractive than the sm...       1  

[40001 rows x 7 columns]

#### Removing Stop words

In [19]:
stop_words = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']

In [20]:
dataset['reviewText'] = dataset['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [21]:
dataset['summary'] = dataset['summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [22]:
dataset

itemName            brand  \
0      Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...        Toblerone   
1      Prince of Peace Organic Tea, Oolong, 100 Tea Bags  Prince Of Peace   
2      Pet Champion Adjustable No-Pull Harness, Colla...     Pet Champion   
3      Parker Urban Premium Ebony Metal Chiseled, Bal...           Parker   
4      Merrick Whole Earth Farms Adult Recipe Dry Dog...          Merrick   
...                                                  ...              ...   
39996         Fluker's Ceramic Heat Emitter for Reptiles         Fluker's   
39997  Purina Friskies Pate Wet Cat Food - (24) 5.5 o...  Purina Friskies   
39998  PetSafe Healthy Pet Water Station, Dog and Cat...          PetSafe   
39999        Alumilite Amazing Mold Rubber Kit, 0.77 oz.        Alumilite   
40000  Our Pets Elevated Bone Feeder Raised Dog Feedi...         Our Pets   

                       category  rating  \
0                  Prime_Pantry     5.0   
1      Grocery_and_Gourmet_Food     5.0   
2                  Pet_Supplies     5.0   
3               Office_Products     5.0   
4                  Pet_Supplies     2.0   
...                         ...     ...   
39996              Pet_Supplies     5.0   
39997              Pet_Supplies     5.0   
39998              Pet_Supplies     1.0   
39999    Arts_Crafts_and_Sewing     5.0   
40000              Pet_Supplies     5.0   

                                                 summary  \
0                                             five stars   
1                                             five stars   
2      has odd shape chest and her pull over harness ...   
3                                          love this pen   
4                                               no bueno   
...                                                  ...   
39996                                         five stars   
39997                                         bon apptit   
39998                                     rust spots ugh   
39999                                         five stars   
40000                              exactly what expected   

                                              reviewText  labels  
0               super smooth and yummy with crunchy bits       1  
1                                   perfect for kombucha       1  
2      finally harness that fits my puppy really was ...       1  
3      love this pen love shape feel weight and color...       1  
4      we accidentally purchased this instead our usu...      -1  
...                                                  ...     ...  
39996                                      working great       1  
39997  this is small rescue shelter and we go through...       1  
39998  ve only had this for couple months and metal d...      -1  
39999  like this stuff think s little pricy but works...       1  
40000  this stand is much more attractive than small ...       1  

[40001 rows x 7 columns]

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
vectorizer = CountVectorizer(analyzer = "word", 
                             preprocessor = None, 
                             stop_words =  'english', 
                             max_features = 6000, ngram_range=(1,5))
data_features = vectorizer.fit_transform(dataset['reviewText'])


In [25]:
tfidf_transformer = TfidfTransformer()
data_features_tfidf = tfidf_transformer.fit_transform(data_features)
data_mat = data_features_tfidf.toarray()

In [26]:
data_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Defining Train and Test partitions

In [27]:
y = np.array(dataset['labels'])

In [28]:
test_index = np.append(np.random.choice((np.where(y==-1))[0], 2000, replace=False),np.random.choice((np.where(y==1))[0], 2000, replace=False))

In [29]:
train_index = list(set(range(len(y))) - set(test_index))

In [30]:
test_index

array([28776, 10671, 35092, ..., 38187, 18790, 34431], dtype=int64)

In [31]:
train_data = data_mat[train_index,]
train_labels = y[train_index]
test_data = data_mat[test_index,]
test_labels = y[test_index]

#### Fit logistic regression classifier on training data
#### Get predictions on training and test data
#### Compute errors

In [32]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(train_data, train_labels)

preds_train = clf.predict(train_data)
preds_test = clf.predict(test_data)
## Compute errors
errs_train = np.sum((preds_train > 0.0) != (train_labels > 0.0))
errs_test = np.sum((preds_test > 0.0) != (test_labels > 0.0))
print("Training error: ", float(errs_train)/len(train_labels))
print("Test error: ", float(errs_test)/len(test_labels))

Training error:  0.14557928946418155
Test error:  0.27025


## Predicting on unseen data
#### Prediction dataset defined at the beginning of the code (10000 records)

In [33]:
prediction_dataset.head()

userName                                           itemName  \
40000     Linda D.  Our Pets Elevated Bone Feeder Raised Dog Feedi...   
40001       maofla  Hill'S Science Diet Adult Optimal Care Chicken...   
40002  kim tindell  goDog Dinos with Chew Guard Technology Durable...   
40003    ItsYvonne  Topenca Supplies 1 Inches x 50 Yards Double Fa...   
40004       Marcia   Sun Flowers Embroidered Iron on Applique Patches   

                                             description  \
40000  ['Elevate your dogs dining experience with the...   
40001  ["Provide your cat trusted, balanced nutrition...   
40002  ['goDog Dinos are a blast from the past.Made w...   
40003  ['The TOPENCA SUPPLIES is a sophisticated sati...   
40004  ['Iron on patches are a fun and simple way to ...   

                                                   image                brand  \
40000  ['https://images-na.ssl-images-amazon.com/imag...             Our Pets   
40001  ['https://images-na.ssl-images-amazon.com/imag...  Hill's Science Diet   
40002  ['https://images-na.ssl-images-amazon.com/imag...                goDog   
40003  ['https://images-na.ssl-images-amazon.com/imag...     Topenca Supplies   
40004                                                 []           Happy Natt   

                                                 feature  \
40000  ['Comfortable dining for your pet', 'Reduces s...   
40001  ["Hill'S Science Diet Adult Optimal Care Healt...   
40002  ['goDog Blue Spike Dino with Chew Guard Techno...   
40003  ['THE BEST CRAFT ACCESSORY People will always ...   
40004  ['Size 3 3/4" wide X 5" tall', 'Top Quality, D...   

                     category   price  rating  reviewTime  \
40000            Pet_Supplies  $34.19     5.0  2018-01-13   
40001            Pet_Supplies  $13.99     5.0  2018-01-13   
40002            Pet_Supplies  $12.44     3.0  2018-01-13   
40003  Arts_Crafts_and_Sewing  $15.99     5.0  2018-01-13   
40004  Arts_Crafts_and_Sewing   $3.68     5.0  2018-01-13   

                         summary  \
40000    Exactly what I expected   
40001                 Five Stars   
40002  Squeaker stopped working!   
40003          pretty and useful   
40004                 Five Stars   

                                              reviewText  vote  
40000  This stand is much more attractive than the sm...     0  
40001  Great nutrition for my cats. They have sensiti...     0  
40002  Puppy likes the toy BUT the squeaker only last...     0  
40003                          happy to get this ribbon.     0  
40004           Bright and happy on my crocheted afghan.     0

In [34]:
len(prediction_dataset["itemName"].unique())

7205

## Preprocessing unseen data (prediction dataset)

#### Converting Text to lower case and string trimming

In [35]:
prediction_dataset['summary'] = prediction_dataset['summary'].astype(str)

In [36]:
prediction_dataset['summary'] = prediction_dataset['summary'].str.lower().str.strip()

In [37]:
prediction_dataset['reviewText'] = prediction_dataset['reviewText'].str.lower().str.strip()

In [38]:
for rating in np.sort(prediction_dataset['rating'].unique()):
    if rating in [1,2,3,4]:
        prediction_dataset.at[prediction_dataset[prediction_dataset['rating'] == rating].index.tolist(),'labels'] = -1
    if rating in [5]:
        prediction_dataset.at[prediction_dataset[prediction_dataset['rating'] == rating].index.tolist(),'labels'] = 1

In [39]:
prediction_dataset['labels'] = prediction_dataset['labels'].astype(int)

#### Removing Punctuation from the Text

In [40]:
prediction_dataset['reviewText'] = prediction_dataset['reviewText'].astype(str)

In [41]:
for punc in list(string.punctuation):
    prediction_dataset['summary'] = prediction_dataset['summary'].apply(lambda x:x.replace(punc,' '))
    prediction_dataset['reviewText'] = prediction_dataset['reviewText'].apply(lambda x:x.replace(punc,' '))

#### Removing Stop words

In [42]:
stop_words = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']

In [43]:
prediction_dataset['reviewText'] = prediction_dataset['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [44]:
prediction_dataset['summary'] = prediction_dataset['summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
vectorizer = CountVectorizer(analyzer = "word", 
                             preprocessor = None, 
                             stop_words =  'english', 
                             max_features = 6000, ngram_range=(1,5))
data_features = vectorizer.fit_transform(prediction_dataset['reviewText'])


In [47]:
tfidf_transformer = TfidfTransformer()
data_features_tfidf = tfidf_transformer.fit_transform(data_features)
data_mat = data_features_tfidf.toarray()

In [48]:
data_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Records where sentiment prediction is high based on text reviews are filtered out 

In [49]:
productIndex= np.where(clf.predict(data_mat)==1)[0]


## Display 5 products among the ones where the sentiment is predicted as positive based on text reviews. These products are selected by the risk-averse investor for selling and investing capital into

In [50]:
prediction_dataset.iloc[productIndex,[1,2,4]][:5]

itemName  \
40000  Our Pets Elevated Bone Feeder Raised Dog Feedi...   
40002  goDog Dinos with Chew Guard Technology Durable...   
40003  Topenca Supplies 1 Inches x 50 Yards Double Fa...   
40005              Scor-Tape 1/4&quot; X 27yds (5 Rolls)   
40006  CROCHET HOOKS SET W/ 3 YARN NEEDLES [UPDATED] ...   

                                             description              brand  
40000  ['Elevate your dogs dining experience with the...           Our Pets  
40002  ['goDog Dinos are a blast from the past.Made w...              goDog  
40003  ['The TOPENCA SUPPLIES is a sophisticated sati...   Topenca Supplies  
40005  ['Premium, double-sided adhesive tape that is ...           Scor-Pal  
40006                                                 []  Athena's Elements